In [1]:
import pandas as pd
import numpy as np
from kmapper import KeplerMapper
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from kmapper.cover import Cover
from sklearn.cluster import DBSCAN

/home/ace/.local/lib/python3.12/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [5]:
#read in big names, row labled by year-author-subgenre-title
df = pd.read_csv("/Users/keliher/Documents/DHUROP26/Data/BigNamesOfScience/BigNamesLabeled.csv", index_col=0)

#drop non-numeric columns
columns_to_drop = ['text_name','text_key', 'html_name', 'chunk_index', '!UNRECOGNIZED', '!UNTAGGED', '!BLACKLISTED', '<# Word Tokens>', '<# Punctuation Tokens>', '<# Tokens>']
df = df.drop(columns=columns_to_drop)

df.head(10)

,AbstractConcepts,Acknowledge,Anger,Apology,Aside,Attack_Citation,Authoritative_Citation,Autobio,Biographical_Time,Cause,...,SubjectivePercept,SubjectiveTime,Substitution,Support,TimeDate,TimeDuration,TimeShift,Transformation,Uncertainty,Updates
1570-Euclid-Mathematics-The elements of geometrie of the most auncient philosopher Euclide,5.880564,0.001389,0.002779,0.000463,0.460985,0.000000,0.001698,0.007565,0.042918,0.076728,...,1.213905,0.035199,0.009109,0.008028,0.001235,0.164571,0.056041,0.246857,0.360173,0.098187
1631-Fludd-Occultism-Doctor Fludds answer vnto MÂ Foster or The squeesing of,4.138300,0.028133,0.109717,0.004220,1.097170,0.000000,0.011253,0.073145,0.129410,0.315085,...,2.599449,0.123783,0.022506,0.087211,0.000000,0.392449,0.191301,0.519046,0.445901,0.351657
1638-Foster-Instruments-The art of dialling by a new easie and most,7.444412,0.000000,0.000000,0.000000,0.372221,0.000000,0.000000,0.019591,0.019591,0.225291,...,0.607307,0.107748,0.000000,0.039181,0.039181,0.959937,0.274268,0.391811,0.215496,0.176315
1619-Bacon-Philosophy of Science-The vvisedome of the ancients written in Latine by the,3.868852,0.025501,0.211293,0.000000,0.834244,0.000000,0.010929,0.018215,0.378871,0.167577,...,2.105647,0.561020,0.010929,0.029144,0.000000,0.586521,0.327869,0.699454,0.947177,0.349727
1586-Galen-Medicine-Treatments-Certaine vvorkes of Galens called Methodus medendi with a briefe,3.915228,0.004244,0.098454,0.000000,1.133914,0.000000,0.001697,0.057714,0.201151,0.310638,...,1.236611,0.249529,0.005941,0.190117,0.000000,0.807999,0.263109,0.528764,1.078746,0.328462
1638-Bacon-Natural History-The historie of life and death With observations naturall and,3.611563,0.005006,0.057565,0.000000,0.230259,0.000000,0.000000,0.055062,0.322863,0.182706,...,2.490302,0.412965,0.052559,0.107621,0.015017,2.139907,0.165186,1.278939,0.498060,0.185208
1638-Bacon-Natural History-Historie naturall and experimentall of life and death Or of,3.014281,0.003195,0.052714,0.000000,1.017539,0.000000,0.000000,0.035143,0.273154,0.146960,...,2.194818,0.416920,0.025558,0.067091,0.015974,1.448836,0.289128,0.771541,0.988786,0.309894
1605-Bacon-Philosophy of Science-The tvvoo bookes of Francis Bacon Of the proficience and,4.838999,0.011602,0.068556,0.000000,1.097951,0.000000,0.008438,0.036915,0.244692,0.229926,...,1.826754,0.255239,0.011602,0.032696,0.004219,0.491494,0.286880,0.491494,0.786812,0.309029
1627-Bacon-Natural History-Sylua syluarum or A naturall historie In ten centuries VVritten,3.717834,0.012528,0.069471,0.000000,0.915650,0.000569,0.001139,0.039291,0.146345,0.607586,...,1.465153,0.273328,0.018222,0.055805,0.013666,0.834790,0.404298,0.857567,0.874081,0.284717
1635-Gellibrand-Mathematics-A discourse mathematical on the variation of the magneticall needle,9.058267,0.000000,0.016321,0.000000,0.391709,0.000000,0.000000,0.000000,0.000000,0.179533,...,0.767096,0.408030,0.000000,0.032642,0.261139,1.077199,0.261139,0.440672,0.603884,0.326424


In [6]:
#scale the data
X = df.select_dtypes(include="number")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X) #take X and scale to [0,1] with std dev 1


In [7]:
#mapper takes np arrays, but we should preserve row names 

X_scaled = pd.DataFrame(
    X_scaled,
    index=df.index,
    columns=df.columns
)

#check it
X_scaled.head(3)


,AbstractConcepts,Acknowledge,Anger,Apology,Aside,Attack_Citation,Authoritative_Citation,Autobio,Biographical_Time,Cause,...,SubjectivePercept,SubjectiveTime,Substitution,Support,TimeDate,TimeDuration,TimeShift,Transformation,Uncertainty,Updates
1570-Euclid-Mathematics-The elements of geometrie of the most auncient philosopher Euclide,0.374638,-0.656612,-1.130280,-0.213132,-0.799031,-0.377559,-0.583792,-0.661981,-0.944424,-1.195027,...,-0.515698,-2.057380,-0.827043,-0.719027,-0.294580,-1.191723,-1.842535,-1.158829,-1.201787,-1.702813
1631-Fludd-Occultism-Doctor Fludds answer vnto MÂ Foster or The squeesing of,-0.702615,0.684729,0.454034,0.500186,1.579108,-0.377559,0.151457,-0.248211,-0.253559,0.348138,...,2.097888,-1.177041,-0.305092,0.245885,-0.301549,-0.839915,-0.909765,-0.384046,-0.956273,0.181335
1638-Foster-Instruments-The art of dialling by a new easie and most,1.341576,-0.726302,-1.171449,-0.301072,-1.130842,-0.377559,-0.714470,-0.586105,-1.130757,-0.233200,...,-1.659940,-1.336397,-1.181901,-0.339398,-0.080469,0.036198,-0.337622,-0.746218,-1.616124,-1.122054


In [8]:
#pca two componenets 
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

print("Variance explained by each principle comp:", pca.explained_variance_ratio_)
print("Cumulative variance:", pca.explained_variance_ratio_.cumsum())

Variance explained by each principle comp: [0.15526954 0.06648878]
Cumulative variance: [0.15526954 0.22175832]


In [9]:
#dimensionality reduction, project to PCA space with two most significant comps
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# X_pca will be used as the lens/ruler 
lens = X_pca

In [10]:
#Init. Kepler Mapper
mapper = KeplerMapper(verbose=1)

KeplerMapper(verbose=1)


In [11]:
#create cover 
cover = Cover(n_cubes=15, perc_overlap=0.1)

In [12]:
# Create Mapper graph
graph = mapper.map(
    lens,
    X_scaled,
    clusterer=DBSCAN(eps=10, min_samples=3),
    cover=cover
)

Mapping on data shaped (329, 115) using lens shaped (329, 2)

Creating 225 hypercubes.

Created 35 edges and 37 nodes in 0:00:00.078548.


In [13]:
tooltips = df.index.astype(str).to_numpy()

mapper.visualize(graph, path_html="/Users/keliher/Documents/DHUROP26/Code/BigNamesNetwork.html",
                 title="Big Names Network!", custom_tooltips=tooltips)

Wrote visualization to: /Users/keliher/Documents/DHUROP26/Code/BigNamesNetwork.html


'<!DOCTYPE html>\n<html>\n\n<head>\n  <meta charset="utf-8">\n  <meta name="generator" content="KeplerMapper">\n  <title>Big Names Network! | KeplerMapper</title>\n\n  <link rel="icon" type="image/png" href="http://i.imgur.com/axOG6GJ.jpg" />\n\n  <link href=\'https://fonts.googleapis.com/css?family=Roboto+Mono:700,300\' rel=\'stylesheet\' type=\'text/css\'>\n  <style>* {\n  margin: 0;\n  padding: 0;\n}\n\nhtml, body {\n  height: 100%;\n}\n\nbody {\n  font-family: "Roboto Mono", "Helvetica", sans-serif;\n  font-size: 14px;\n}\n\n#logo {\n  width:  85px;\n  height: 85px;\n}\n\n#display {\n  color: #95A5A6;\n  background: #212121;\n}\n\n#header {\n  background: #111111;\n}\n\n#print {\n  color: #000;\n  background: #FFF;\n}\n\nh1 {\n  font-size: 21px;\n  font-weight: 300;\n  font-weight: 300;\n}\n\nh2 {\n  font-size: 18px;\n  padding-bottom: 20px;\n  font-weight: 300;\n}\n\nh3 {\n  font-size: 14px;\n  font-weight: 700;\n  text-transform: uppercase;\n}\n\nh4 {\n  font-size: 13px;\n  font-